In [1]:
%%capture

!pip install gdown

Analyze screen time on MacOS to see where I spend my time:

```
cd ~/gitRepo/ScreenTime2CSV/ScreenTime2CSV
python3 screentime2csv.py -o output.tsv -d '\t'
```

In [2]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/1aXTzmjvdvLesw1MH26wAL2UGC1EjL0sn/view?usp=sharing'
gdown.download(URL, 'output.tsv', quiet=True, fuzzy=True)

In [25]:
import pandas as pd

df = pd.read_csv('output.tsv', sep='\t')
df.head(3)

,app,usage,start_time,end_time,created_at,tz,device_id,device_model
0,com.apple.systempreferences,6,1736824855,1736824861,1.736825e+09,25200,NaN,NaN
1,com.apple.Terminal,1,1736824854,1736824855,1.736825e+09,25200,NaN,NaN
2,com.apple.systempreferences,12,1736824842,1736824854,1.736825e+09,25200,NaN,NaN


In [43]:
df['date'] = pd.to_datetime(df['created_at'], unit='s').dt.strftime('%d-%m-%Y')
df['datetime'] = pd.to_datetime(df['created_at'], unit='s', utc=True).dt.tz_convert('Asia/Ho_Chi_Minh')

df = df.sort_values(by='datetime', ascending=False)
df = df[df['datetime'].dt.weekday < 5]

df.head(3)

,app,usage,start_time,end_time,created_at,tz,device_id,device_model,date,datetime,prev_time,gap_minutes,block
8008,com.google.Chrome,9,1741662172,1741662181,1.741662e+09,25200,NaN,NaN,11-03-2025,2025-03-11 10:03:01.273828983+07:00,2025-03-11 03:02:52.834475994,0.140656,51
8009,com.apple.Terminal,3,1741662169,1741662172,1.741662e+09,25200,NaN,NaN,11-03-2025,2025-03-11 10:02:52.834475994+07:00,2025-03-11 03:02:49.210155010,0.060405,51
8010,com.apple.finder,2,1741662167,1741662169,1.741662e+09,25200,NaN,NaN,11-03-2025,2025-03-11 10:02:49.210155010+07:00,2025-03-11 03:02:47.238322020,0.032864,51


In [35]:
daily_usage = df.groupby(['date'])['usage'].sum().reset_index()

daily_usage['hours'] = (daily_usage['usage'] // 3600).astype(int)
daily_usage['minutes'] = ((daily_usage['usage'] % 3600) // 60).astype(int)

daily_usage['datetime'] = pd.to_datetime(df['date'], format="%d-%m-%Y")
daily_usage['day_of_week'] = daily_usage['datetime'].dt.strftime('%A')

daily_usage['sort_date'] = pd.to_datetime(daily_usage['date'], format='%d-%m-%Y')
daily_usage = daily_usage.sort_values(by='sort_date', ascending=False)

median_usage = daily_usage['usage'].median()

median_hours = int(median_usage // 3600)
median_minutes = int((median_usage % 3600) // 60)

print(f"\nMedian: {median_hours}h:{median_minutes}m\n")

for index, row in daily_usage.iterrows():
    print(f"{row['date']}: {row['hours']}h:{row['minutes']}m")


Median: 5h:31m

11-03-2025: 0h:51m
10-03-2025: 7h:31m
07-03-2025: 10h:47m
06-03-2025: 9h:29m
05-03-2025: 5h:36m
04-03-2025: 8h:20m
03-03-2025: 6h:27m
28-02-2025: 5h:5m
27-02-2025: 8h:54m
26-02-2025: 5h:19m
25-02-2025: 6h:19m
24-02-2025: 7h:37m
21-02-2025: 4h:46m
20-02-2025: 5h:19m
19-02-2025: 5h:35m
18-02-2025: 4h:2m
14-01-2025: 1h:2m
13-01-2025: 4h:56m
10-01-2025: 6h:14m
09-01-2025: 4h:23m
08-01-2025: 5h:31m
07-01-2025: 6h:39m
06-01-2025: 9h:7m
03-01-2025: 6h:26m
02-01-2025: 5h:46m
01-01-2025: 0h:35m
25-12-2024: 4h:23m
24-12-2024: 3h:39m
23-12-2024: 2h:46m
20-12-2024: 5h:0m
19-12-2024: 4h:0m
18-12-2024: 1h:20m
17-12-2024: 6h:3m


In [44]:
import pytz

df = df.sort_values(by=['date', 'datetime'])

# Compute time difference from previous entry
df['prev_time'] = df.groupby('date')['datetime'].shift(1)
df['gap_minutes'] = (df['datetime'] - df['prev_time']).dt.total_seconds() / 60

# Identify activity blocks
df['block'] = (df['gap_minutes'] >= 30).cumsum()  # New block starts if gap ≥ 30 minutes

# Compute start and end time for each block
activity_blocks = df.groupby(['date', 'block']).agg(
    start_time=('datetime', 'first'),
    end_time=('datetime', 'last')
).reset_index()

# Compute duration of each block
activity_blocks['duration_minutes'] = (activity_blocks['end_time'] - activity_blocks['start_time']).dt.total_seconds() / 60

# Keep only blocks that last at least 30 minutes
valid_blocks = activity_blocks[activity_blocks['duration_minutes'] >= 30]

for date, group in valid_blocks.groupby('date'):
    print(f"\nDate: {date}")
    for _, row in group.iterrows():
        start = row['start_time'].strftime('%H:%M')
        end = row['end_time'].strftime('%H:%M')
        duration = int(row['duration_minutes'])
        print(f"  - {start} to {end} ({duration} min)")



Date: 01-01-2025
  - 16:24 to 16:54 (30 min)
  - 20:02 to 20:34 (32 min)

Date: 02-01-2025
  - 08:57 to 11:29 (151 min)
  - 12:10 to 15:46 (216 min)

Date: 03-01-2025
  - 09:41 to 15:46 (364 min)
  - 17:18 to 18:34 (75 min)

Date: 03-03-2025
  - 09:14 to 11:33 (139 min)
  - 12:11 to 15:47 (215 min)
  - 16:39 to 17:21 (41 min)

Date: 04-03-2025
  - 09:15 to 15:45 (389 min)
  - 17:16 to 18:54 (98 min)
  - 23:17 to 23:53 (36 min)

Date: 05-03-2025
  - 09:42 to 10:15 (32 min)
  - 10:52 to 13:00 (128 min)
  - 13:31 to 15:38 (126 min)
  - 16:14 to 18:18 (124 min)

Date: 06-01-2025
  - 09:03 to 17:09 (485 min)
  - 18:14 to 21:48 (214 min)

Date: 06-03-2025
  - 09:33 to 15:53 (380 min)
  - 16:38 to 19:18 (159 min)
  - 20:28 to 22:46 (137 min)

Date: 07-01-2025
  - 08:42 to 15:39 (416 min)
  - 16:59 to 17:46 (47 min)
  - 18:20 to 19:09 (48 min)

Date: 07-03-2025
  - 08:00 to 08:46 (46 min)
  - 09:19 to 13:57 (278 min)
  - 14:37 to 15:44 (67 min)
  - 17:08 to 19:09 (121 min)

Date: 08-01-2025
 